In [5]:
from pytube import YouTube
from tqdm import tqdm
import cv2
from matplotlib import pyplot as plt
import pytesseract
import numpy as np
from PIL import Image
import copy
import os

In [6]:
# Video to process
game_name = "clg_ggs_w6d2_s2019"
highlights_yt = 'https://www.youtube.com/watch?v=YRAgGKrPVLc'
game_yt = 'https://www.youtube.com/watch?v=_FGBtmfXzj0'

In [7]:
# Add folders
if not os.path.exists("frames"):
    os.makedirs("frames")
if not os.path.exists("source"):
    os.makedirs("source")

In [8]:
# highlights
yt = YouTube(highlights_yt)

stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

pbar = tqdm(total=stream.filesize)
def progress_fn(self, chunk, *_):
    pbar.update(len(chunk))
yt.register_on_progress_callback(progress_fn)    

stream.download(output_path="./source/" + game_name, filename="source_high_res_highlights") 

KeyError: 'url_encoded_fmt_stream_map'

In [22]:
# video
yt = YouTube(game_yt)

stream = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()

pbar = tqdm(total=stream.filesize)

yt.register_on_progress_callback(progress_fn)    

stream.download(output_path="./source/" + game_name, filename="src") 


100%|█████████▉| 758775808/759152504 [01:35<00:00, 9482044.06it/s]

'./source/clg_ggs_w6d2_s2019/src.mp4'


100%|██████████| 759152504/759152504 [01:49<00:00, 9482044.06it/s]

In [38]:
fps = stream.fps

In [23]:
# Crop image to get timestamp

def get_num_from_img(crop_img, blur_size=None):

    crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    flip_img = 255 - crop_img

    # ret, thresh = cv2.threshold(flip_img, 127, 255, cv2.THRESH_BINARY) 
    ret, thresh = cv2.threshold(flip_img, 170, 255, cv2.THRESH_BINARY) 

    upscale_img = cv2.resize(thresh, (thresh.shape[1] * 20, thresh.shape[0] * 20), interpolation = cv2.INTER_AREA) 
    
    if not blur_size:
        blur_size = 25
        
    blur = cv2.blur(upscale_img,(blur_size,blur_size))
        
    
    pil_img = Image.fromarray(blur)
    text = pytesseract.image_to_string(pil_img,  config='outputbase digits')
        
    kernel_sharpening = np.array([[-1,-1,-1], 
                                  [-1, 9,-1],
                                  [-1,-1,-1]])
    
    sharpened = cv2.filter2D(blur, -1, kernel_sharpening)
    pil_img = Image.fromarray(sharpened)
    if text == "":
        text = pytesseract.image_to_string(pil_img,  config='outputbase digits')

    
    pil_img.save("test.jpg", "JPEG")
    
#     if (text == "" and blur_size == None):
# #     0.8571428571428571
#         return (get_num_from_img(crop_img. blur_size + 10))
    
#     if text != "":
#         plt.figure()
#         plt.subplot(161),plt.imshow(crop_img),plt.title('Original')
#         plt.subplot(162),plt.imshow(flip_img),plt.title('Flipped')
#         plt.subplot(163),plt.imshow(thresh),plt.title('Threshold')
#         plt.subplot(164),plt.imshow(upscale_img),plt.title('Upscaled')
#         plt.subplot(165),plt.imshow(blur),plt.title('Blur ' + text)
#         plt.subplot(166),plt.imshow(sharpened),plt.title('Sharpened ' + text)
    return text

def get_timestamp_from_img(img):
    height, width, chan = img.shape

    crop_img = img[int(height * 0.068): int(height * 0.1), int(width * 0.485) : int(width * 0.52)]
    crop_img_left = img[int(height * 0.07): int(height * 0.095), int(width * 0.49) : int(width * 0.503)]
    crop_img_left = np.pad(crop_img_left, [(5, 5), (5, 5), (0,0)], mode='constant')
    crop_img_right = img[int(height * 0.07): int(height * 0.09), int(width * 0.504) : int(width * 0.515)]
    crop_img_right = np.pad(crop_img_right, [(5, 5), (5, 5), (0,0)], mode='constant')
    
    left = get_num_from_img(crop_img_left)
    try:
        left = int(left)
    except:
        left = ""
    right = get_num_from_img(crop_img_right)
    try:
        right = int(right)
    except:
        right = ""
    
    return (left, right)

def time_to_sec(time):
    minute, sec = time[0], time[1]
    return minute * 60 + sec
        

In [26]:
# Get the raw times
times = []
vidcap = cv2.VideoCapture('source/' + game_name + '/source_high_res_highlights.mp4')
count = 0
success = True
while success:
    success,image = vidcap.read()
    
    if count % 40 == 0:
        (left, right) = get_timestamp_from_img(image)
        times.append((left, right))
        print((left, right))

    count += 1

print(times)


('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
('', '')
(9, '')
(9, '')
(9, '')
('', '')
('', '')
('', '')
('', 24)
(5, 23)
(5, 24)
(5, 26)
(5, 27)
(5, '')
(5, 30)
('', 31)
(5, 33)
(5, 34)
(5, 35)
(5, 37)
(5, 38)
(5, 0)
(5, 41)
(5, 41)
(7, 15)
(7, 17)
(7, 18)
(7, 20)
(7, 21)
(7, 22)
(7, 24)
(7, 59)
(8, 0)
(8, 1)
(8, 3)
(8, 4)
(8, 6)
(8, 7)
(8, 8)
(8, 9)
(8, 10)
('', 12)
(8, 13)
(8, 14)
(8, 28)
('', 29)
(8, 30)
(8, 32)
(8, 33)
(8, 34)
('', 36)
('', 37)
('', 38)
('', 40)
(0, 3)
(10, 4)
(10, 6)
(10, 7)
('', 9)
(10, 10)
(10, 11)
(10, 13)
(10, 14)
(12, 7)
(12, 3)
(12, 10)
(12, 11)
(12, 12)
(12, 14)
(12, 15)
(12, 17)
(12, 18)
(12, 19)
(12, 21)
(12, 22)
(12, 23)
(12, 24)
(12, 25)
(12, 27)
(12, 28)
(12, 30)
(12, 31)
(12, 32)
(12, 34)
(12, 35)
(13, 29)
(13, 30)
(13, 32)
(13, 33)
(13, 35)
(13, 36)
(13, 37)
(13, 39)
(13, '')
(13, 42)
(13, 43)
(13, '')
(13, 46)
(13, 47)
(13, 49)
(13, 49)
(15, 48)
(15, 49)
(15, 51)
(15, 52)
(15, 53)
(15, 55)
(15, 56)
(15, 57)
(15, 58)
(15, 

In [27]:
# Filter the first empty times (from non highlight footage)
saw_non_empty = False
times_first_non_empty = []
for time in times:
    if time[0] != "" and time[1] != "":
        saw_non_empty = True
    if time[0] == "" and time[1] == "":
        continue
    if saw_non_empty:
        times_first_non_empty.append(time)

last_time = None
# Guess the empty times
times_non_empty_temp = []
for time in times_first_non_empty:
    
    left = time[0]
    right = time[1]
    
    if last_time == None:
        last_time = [time[0], time[1]]
    else:
        # Going down in minutes, skip
        if left != "" and left < last_time[0]:
            continue
        if left != "":
            last_time[0] = left
        # New time but need to incr the minute
        if right != "" and right < last_time[1] and right < 5:
            last_time[0] += 1
        if right != "":
            last_time[1] = right
            
    times_non_empty_temp.append(copy.deepcopy(last_time))
    
# Filter to make sure none are empty
times_non_empty = []
for time in times_non_empty_temp:
    if (time[0] != "" and time[1] != ""):
        times_non_empty.append(time)
   
print(times_non_empty)


[[5, 23], [5, 24], [5, 26], [5, 27], [5, 27], [5, 30], [5, 31], [5, 33], [5, 34], [5, 35], [5, 37], [5, 38], [6, 0], [7, 15], [7, 17], [7, 18], [7, 20], [7, 21], [7, 22], [7, 24], [7, 59], [9, 0], [9, 12], [9, 29], [9, 36], [9, 37], [9, 38], [9, 40], [11, 4], [11, 9], [12, 7], [13, 3], [13, 29], [13, 30], [13, 32], [13, 33], [13, 35], [13, 36], [13, 37], [13, 39], [13, 39], [13, 42], [13, 43], [13, 43], [13, 46], [13, 47], [13, 49], [13, 49], [15, 48], [15, 49], [15, 51], [15, 52], [15, 53], [15, 55], [15, 56], [15, 57], [15, 58], [15, 59], [17, 1], [17, 2], [17, 6], [17, 31], [18, 1], [18, 47], [18, 48], [18, 50], [18, 51], [18, 52], [18, 54], [18, 55], [18, 57], [18, 58], [18, 59], [20, 1], [22, 9], [22, 9], [22, 12], [22, 14], [22, 15], [22, 16], [22, 18], [22, 18], [22, 20], [22, 21], [22, 22], [22, 24], [22, 25], [22, 27], [22, 28], [22, 29], [22, 31], [22, 32], [22, 34], [22, 35], [22, 55], [22, 57], [22, 53], [24, 0], [24, 0], [24, 2], [24, 3], [24, 5], [24, 6], [24, 7], [24, 8]

In [28]:
# Remove extremes
non_extreme_times = []
for i in range(len(times_non_empty)):
    left_diff = 0
    if i-1 > 0:
        left_diff = abs(time_to_sec(times_non_empty[i]) - time_to_sec(times_non_empty[i-1]))
    right_diff = 0
    if i+1 < len(times_non_empty):
        right_diff =  abs(time_to_sec(times_non_empty[i]) - time_to_sec(times_non_empty[i+1]))
    if left_diff > 10 and right_diff > 10:
        continue
    
    non_extreme_times.append(times_non_empty[i])
print(non_extreme_times)

[[5, 23], [5, 24], [5, 26], [5, 27], [5, 27], [5, 30], [5, 31], [5, 33], [5, 34], [5, 35], [5, 37], [5, 38], [7, 15], [7, 17], [7, 18], [7, 20], [7, 21], [7, 22], [7, 24], [9, 29], [9, 36], [9, 37], [9, 38], [9, 40], [11, 4], [11, 9], [13, 29], [13, 30], [13, 32], [13, 33], [13, 35], [13, 36], [13, 37], [13, 39], [13, 39], [13, 42], [13, 43], [13, 43], [13, 46], [13, 47], [13, 49], [13, 49], [15, 48], [15, 49], [15, 51], [15, 52], [15, 53], [15, 55], [15, 56], [15, 57], [15, 58], [15, 59], [17, 1], [17, 2], [17, 6], [18, 47], [18, 48], [18, 50], [18, 51], [18, 52], [18, 54], [18, 55], [18, 57], [18, 58], [18, 59], [22, 9], [22, 9], [22, 12], [22, 14], [22, 15], [22, 16], [22, 18], [22, 18], [22, 20], [22, 21], [22, 22], [22, 24], [22, 25], [22, 27], [22, 28], [22, 29], [22, 31], [22, 32], [22, 34], [22, 35], [22, 55], [22, 57], [22, 53], [24, 0], [24, 0], [24, 2], [24, 3], [24, 5], [24, 6], [24, 7], [24, 8], [24, 9], [24, 11], [24, 12], [24, 13], [24, 15], [24, 16], [24, 17], [24, 19],

In [29]:
timestamps = []

start_time = None
last_time = None
for i in range(len(non_extreme_times)):
    
    time = non_extreme_times[i]
    
    # Start of the highlight
    if start_time == None:
        start_time = time
    
    # Check if this is end of highlight
    if last_time != None and time_to_sec(time) - time_to_sec(last_time) > 10 or i == len(non_extreme_times)-1:
        timestamps.append((copy.copy(start_time), copy.copy(last_time)))
        start_time = copy.copy(time)
    
    last_time = time

print(timestamps)

[([5, 23], [5, 38]), ([7, 15], [7, 24]), ([9, 29], [9, 40]), ([11, 4], [11, 9]), ([13, 29], [13, 49]), ([15, 48], [15, 59]), ([17, 1], [17, 6]), ([18, 47], [18, 59]), ([22, 9], [22, 35]), ([22, 55], [22, 53]), ([24, 0], [24, 19]), ([26, 40], [26, 59]), ([29, 32], [29, 40]), ([30, 0], [30, 8]), ([30, 50], [31, 1])]


In [30]:
# Remove small timestamps and convert to sec
filtered_timestamps = []

for timestamp in timestamps:
    if (time_to_sec(timestamp[1]) - time_to_sec(timestamp[0])) > 5:
        filtered_timestamps.append(range(time_to_sec(timestamp[0]), time_to_sec(timestamp[1])))
print(filtered_timestamps)

[range(323, 338), range(435, 444), range(569, 580), range(809, 829), range(948, 959), range(1127, 1139), range(1329, 1355), range(1440, 1459), range(1600, 1619), range(1772, 1780), range(1800, 1808), range(1850, 1861)]


In [41]:
game_start_in_video_time = time_to_sec([7, 50]) - 8
print(game_start_in_video_time)

vidcap = cv2.VideoCapture('source/' + game_name + '/src.mp4')

print(vidcap.get(cv2.CAP_PROP_FPS))
fps = round(vidcap.get(cv2.CAP_PROP_FPS))
print('fps', fps)
count = 0
success = True

while success:
    success,image = vidcap.read()    
    
    if count % fps == 0:
        game_time = (count / fps) - game_start_in_video_time

        
        is_highlight = False
        print(game_time)
        for r in filtered_timestamps:
            if game_time in r:
                is_highlight = True
                break
            
        if is_highlight:
            cv2.imwrite('frames/' + game_name + '/frame%d-h.jpg' % (count / fps), image)

        else:
            cv2.imwrite('frames/' + game_name + '/frame%d-n.jpg' % (count / fps), image)

    count += 1


462
-462.0
-461.0
-460.0
-459.0
-458.0
-457.0
-456.0
-455.0
-454.0
-453.0
-452.0
-451.0
-450.0
-449.0
-448.0
-447.0
-446.0
-445.0
-444.0
-443.0
-442.0
-441.0


KeyboardInterrupt: 

In [115]:
game_start_in_video_time = time_to_sec([14, 20])
print(game_start_in_video_time) - 10

860
